# Implementing PWM with GPIO using MicroBlaze

In [1]:
import time
import asyncio
import threading
from datetime import datetime
from pynq.lib import LED

In [2]:
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")

## Defining functions to write, read, & reset GPIO pins

In [3]:
%%microblaze base.PMODB

#include "gpio.h"
#include "pyprintf.h"

gpio bgr[3];

// initialize and configure GPIO pins for output
unsigned int init_gpio() {
    bgr[0] = gpio_open(1);
    gpio_set_direction(bgr[0], GPIO_OUT);
    bgr[1] = gpio_open(2);
    gpio_set_direction(bgr[1], GPIO_OUT);
    bgr[2] = gpio_open(3);
    gpio_set_direction(bgr[2], GPIO_OUT);
    return 0;
}

//Function to turn on/off a selected pin of PMODB
unsigned int write_gpio(unsigned int pin, unsigned int val) {
    if (pin < 1 || pin > 3)
        return 1;
    gpio_write(bgr[pin-1], val);
    return 0;
}

//Function to read the value of a selected pin of PMODB
unsigned int read_gpio(unsigned int pin){
    gpio pin_in = gpio_open(pin);
    gpio_set_direction(pin_in, GPIO_IN);
    return gpio_read(pin_in);
}

//Function to reset GPIO pins
unsigned int reset_gpio() {
    write_gpio(1, 0); // blue
    write_gpio(2, 0); // green
    write_gpio(3, 0); // red
    return 0;
}

In [4]:
init_gpio()

0

In [5]:
import time
import random

cond = True
blink_conds = [False for _ in range(5)]
locks = [threading.Lock() for _ in range(5)]        

def blink_fast(num):
    global blink_conds, base
    for i in range(5):
        base.leds[num].on()
        time.sleep(0.1)
        base.leds[num].off()
        time.sleep(0.1)
    base.leds[num].off()
        
def blink_slow(num):
    global blink_conds, base
    for i in range(5):
        base.leds[num].on()
        time.sleep(1.0)
        base.leds[num].off()
        time.sleep(1.0)
    base.leds[num].off()

def blink_fast_pmod():
    for i in range(5):
        write_gpio(1, 1)
        time.sleep(0.1)
        write_gpio(1, 0)
        time.sleep(0.1)

def blink_slow_pmod():
    for i in range(5):
        write_gpio(1, 1)
        time.sleep(1.0)
        write_gpio(1, 0)
        time.sleep(1.0)

def button_handler():
    global button, cond, blink_conds
    while True:
        button = btns.read()
        if button != 0:
            cond = False
        time.sleep(0.01)

In [6]:
def philosopher(num, fork1, fork2):
    global locks, cond, blink_conds
    print(f'Philosopher {num} dispatched')
    acquired = 0    
    while cond:
        # thinks, random sleep
        time.sleep(random.randint(200,800) / 1000)
        # is hungry
        blink_slow(num)
        # acquire both locks
        # eats 
        fork1.acquire(True)
        fork2.acquire(True)
        blink_fast(num)
        fork2.release()
        fork1.release()
        # sleeps
        time.sleep(random.randint(200,800) / 1000)
        
def pmod_philosopher(num, fork1, fork2):
    global locks, cond, blink_conds
    print(f'Philosopher {num} dispatched')
    acquired = 0    
    while cond:
        # thinks, random sleep
        time.sleep(random.randint(200,800) / 1000)
        # is hungry
        blink_slow_pmod()
        # acquire both locks
        # eats 
        fork1.acquire(True)
        fork2.acquire(True)
        blink_fast_pmod()
        fork2.release()
        fork1.release()
        # sleeps
        time.sleep(random.randint(200,800) / 1000)

                
def main_loop():
    # listen for button press
    threads = []
    threads.append(threading.Thread(target=philosopher, args=(0,locks[0], locks[1])))
    threads.append(threading.Thread(target=philosopher, args=(1,locks[1], locks[2])))
    threads.append(threading.Thread(target=philosopher, args=(2,locks[2], locks[3])))
    threads.append(threading.Thread(target=philosopher, args=(3,locks[3], locks[4])))
    threads.append(threading.Thread(target=pmod_philosopher, args=(4, locks[4], locks[0])))
    threads.append(threading.Thread(target=button_handler, args=()))
    
    [t.start() for t in threads]
    # dispath pmod LED

main_loop()

Philosopher 0 dispatched
Philosopher 1 dispatched
Philosopher 2 dispatched
Philosopher 3 dispatched
Philosopher 4 dispatched


Exception in thread Thread-10 (button_handler):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1009, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 946, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_10859/660398069.py", line 43, in button_handler
NameError: name 'btns' is not defined. Did you mean: 'bytes'?
